In [2]:
from math_verify import parse, verify
import re
import pandas as pd

In [3]:
df = pd.read_parquet("../../open-math-reasoning/sample_c7_300_results_repeat_16.parquet")

In [9]:
def extract_last_boxed_greedy(text):
    if not text:
        return None

    positions = [m.start() for m in re.finditer(r'\\boxed\{', text)]
    if not positions:
        return None

    last_pos = positions[-1]
    rest_of_text = text[last_pos:]
    match = re.match(r'\\boxed\{((?:[^{}]|\{[^{}]*\})*)\}', rest_of_text)
    if match:
        return match.group(1)

    return None

In [10]:
extract_last_boxed_greedy(df.response[0])

'\nF = \\mathbb{R} \\setminus \\left( \\bigcup_{i=1}^\\infty (q_i - 2^{-i} \\cdot \\epsilon, q_i + 2^{-i} \\cdot \\epsilon) \\right)\n'

In [15]:
df.expected_answer[0]

'\\( F = \\mathbb{R} \\setminus \\bigcup U_n \\)'

In [16]:
gold = parse(df.expected_answer[0])
answer = parse(extract_last_boxed_greedy(df.response[0]))

# Order here is important!
verify(gold, answer)

False

In [28]:
problem_to_pass_rate = {}
for problem in df.problem.unique():
    df_problem = df[df.problem == problem]
    gold = parse(df_problem.expected_answer.tolist()[0])
    num_pass = 0
    num_total = 0
    for response in df_problem.response:
        num_total += 1
        if verify(gold, parse(extract_last_boxed_greedy(response))):
            num_pass += 1
    problem_to_pass_rate[problem] = num_pass / num_total

In [29]:
problem_to_pass_rate

{'Given the characteristic function \\( f = \\chi_{\\mathbb{Q}} \\) (where \\( \\chi_{\\mathbb{Q}} \\) is the indicator function for the rational numbers), and given \\( \\epsilon > 0 \\), find a closed set \\( F \\subseteq \\mathbb{R} \\) such that \\( m(\\mathbb{R} - F) < \\epsilon \\) and \\( f \\) is continuous on \\( F \\).': 0.0,
 'Calculate the determinant of the matrix \\(\\begin{pmatrix}0&1&2&3&...&n\\\\ 1&0&1&2&...&n-1\\\\ 2&1&0&1&...&n-2\\\\ 3&2&1&0&..&n-3\\\\ ...&...&...&...&...&...\\\\ n&n-1&n-2&n-3&...&0\\end{pmatrix}\\).': 0.0,
 'What is the probability that three pieces of rope, obtained by making two random cuts on a rope, can form a triangle?': 1.0,
 'Evaluate the limit: $\\lim_{n\\to\\infty}\\left(\\frac{1}{\\ln n} \\cdot \\sum_{1 \\le i < j \\le n} \\frac{1}{ij} - \\ln \\sqrt{n}\\right)$.': 0.0,
 'Given the sequence of polynomials \\( L_k(x) \\) defined by \\( L_0(x) = 1 \\), \\( L_1(x) = 1 + x \\), and \\( L_k(x) = L_{k-1}(x) + xL_{k-2}(x) \\) for \\( k \\geq 2 \\)